In [42]:
import pandas as pd

In [43]:
def save(year, train_df, internal_train_df, internal_val_df, dev_df, test_df):
    train_df.to_csv(f"datasets/WASSA{year}_essay_level_train_preproc.tsv", index=False, sep='\t')
    internal_train_df.to_csv(f"datasets/WASSA{year}_essay_level_internal_train_preproc.tsv", index=False, sep='\t')
    internal_val_df.to_csv(f"datasets/WASSA{year}_essay_level_internal_val_preproc.tsv", index=False, sep='\t')
    dev_df.to_csv(f"datasets/WASSA{year}_essay_level_dev_preproc.tsv", index=False, sep='\t')

def read(year):
    train_path = f"datasets/WASSA{year}_essay_level_train_preproc.tsv"
    train_df = pd.read_csv(train_path, sep='\t')
    internal_train_path = f"datasets/WASSA{year}_essay_level_internal_train_preproc.tsv"
    internal_val_path = f"datasets/WASSA{year}_essay_level_internal_val_preproc.tsv"
    internal_train_df = pd.read_csv(internal_train_path, sep='\t')
    internal_val_df = pd.read_csv(internal_val_path, sep='\t')

    dev_path = f"datasets/WASSA{year}_essay_level_dev_preproc.tsv"
    dev_df = pd.read_csv(dev_path, sep='\t')
    return train_df, internal_train_df, internal_val_df, dev_df

def lower_case_labels(dataset):
    for idx, row in dataset.iterrows():
        row['emotion'] = row['emotion'].lower()
    return dataset

In [44]:
def labels(year):
    train_df, internal_train_df, internal_val_df, dev_df = read(year)

    internale_train_df = low_labels(internal_train_df)
    internal_val_df = low_labels(internal_val_df)
    dev_df = low_labels(dev_df)
    train_df = low_labels(train_df)

    save(year, train_df, internal_train_df, internal_val_df, dev_df, test_df)

In [45]:
labels(year=23)
labels(year=22)

In [1]:
'e, ciao .'.split()

['e,', 'ciao', '.']

In [15]:
text = 'It is sad that children are still dying from measles.   According to UNICEF, there is a simple vaccine that can be given, which would stop children from being affected.  It affects most children in poor countries, which is very sad.  The vaccine isn’t that expensive and you would think that the leaders of all countries would get together and do something about this problem.  It is not just up to the USA to stand up and do something.  These other countries (even though they are poor) need to get more educated and help protect their own children and citizens in their own country.  It is sad that children and others have to die because we can’t coordinate with each other, to get the vaccines to those who need it most.  I feel sorry for those families who have suffered the loss of a child due '

In [23]:
split = text.split()

In [1]:
from textblob import TextBlob

In [3]:
s = " -- "
print(s.split())
for si in s.split():
    blob = TextBlob(s)
    print([w.lemmatize() for w in blob.words])

['--']
[]


In [4]:
empathy_values = [] 
distress_values = []
for emp_value, dis_value in zip(empathy_values, distress_values):
    print(emp_value, dis_value)

In [6]:
import pandas as pd

In [20]:
original_train_path = "./datasets/WASSA23_essay_level_original_train_preproc.tsv"
df = pd.read_csv(original_train_path, sep='\t')

In [21]:
df

,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,...,surprise_count,positive_count,negative_count,sadness_count,disgust_count,joy_count,hope_count,empathy_count,distress_count,prompt
0,193,313,It's so sad the turning event of Immigrants st...,6.333333,6.125,36,1,7,1,46,...,0.000000,0.300000,0.300000,0.300000,0.000000,0.000000,0.000000,2.844273,2.460736,"An essay written by a 46 years old female, pos..."
1,443,7,Living in a war tore country must be horrible....,6.666667,6.500,30,1,6,3,37,...,0.028571,0.085714,0.228571,0.085714,0.085714,0.085714,0.117647,3.222620,2.990048,"An essay written by a 37 years old female, pos..."
2,430,397,This was sad and I hope they can do something ...,1.500000,1.000,24,2,7,1,38,...,0.031250,0.125000,0.187500,0.125000,0.062500,0.093750,0.125000,2.921053,2.882645,"An essay written by a 38 years old female, pos..."
3,437,308,This article really brings you back down to ea...,5.000000,5.000,53,2,3,1,27,...,0.040000,0.240000,0.160000,0.160000,0.080000,0.040000,0.052632,2.961519,2.776587,"An essay written by a 27 years old female, pos..."
4,48,210,Reading about the attack on Paris that happene...,1.000000,2.125,23,1,4,2,23,...,0.031250,0.156250,0.156250,0.062500,0.031250,0.125000,0.187500,2.945064,2.887750,"An essay written by a 23 years old female, pos..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,430,397,"After reading the article, i felt really alarm...",1.000000,1.625,17,1,6,1,29,...,0.000000,0.125000,0.312500,0.187500,0.000000,0.062500,0.000000,2.865683,2.804874,"An essay written by a 29 years old female, pos..."
775,374,367,It's very difficult to fathom living somewhere...,2.500000,1.500,24,2,7,1,38,...,0.057143,0.142857,0.200000,0.171429,0.057143,0.028571,0.000000,2.990842,2.915323,"An essay written by a 38 years old female, pos..."
776,58,72,it's crazy how elephant poaching is still a th...,2.666667,1.750,46,1,6,1,34,...,0.000000,0.083333,0.250000,0.250000,0.083333,0.000000,0.000000,2.589131,2.662351,"An essay written by a 34 years old female, pos..."
777,322,52,I just read an article about a Korean man who ...,6.333333,5.625,25,1,4,1,29,...,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,2.740691,2.603962,"An essay written by a 29 years old female, pos..."


In [27]:
for idx, row in df.iterrows():
    if row['essay_id'] == 1791:
        print(row['essay'])
        break

Smugglers are causing so many deaths! You must read this article about their disregard for life and how its causing so many deaths around the globe! So many are trying to escape war, and the governments of these country are giving these smugglers so much freedom -- Its so sad! You have to read this article to belive it!


In [28]:
row

conversation_id                                                                241
article_id                                                                      19
essay                            Smugglers are causing so many deaths! You must...
empathy                                                                   3.333333
distress                                                                     2.875
speaker_id                                                                       3
gender                                                                           2
education                                                                        6
race                                                                             6
age                                                                             55
income                                                                       70000
personality_conscientiousness                                                  4.5
pers

In [ ]:
get_stemmed_EMP_lexicon_per_word(row)